## Importing Pakages

In [2]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import *
from pyspark.sql.functions import trim
import pandas as pd
import numpy as np
import json
import requests
from datetime import datetime, timedelta
import pyspark

In [3]:
pyspark.__version__

'3.2.3'

## Starting Spark Session

In [4]:
spark = SparkSession.builder.appName('CNEO_data_extractor').getOrCreate()
spark

In [5]:
# date = datetime.today().date()
# date_min = date + timedelta(days=59)
# date_max = date + timedelta(days=60)
# print(date_min, date_max)

## Importing Previous/Old/Available Data

In [6]:
sdfl = spark.read.csv('./raw_input_data.csv', inferSchema=True, header=True)
sdfl.show(2)

+-----------+--------+----------------------+-------------------+-----------------------------+--------------------------------+--------------------------------+-----------------+-----------------+-----------------------------+------------------------+-------------+-------------------+-----------------+
|Designation|Orbit Id|Time of Close approach|Close-Approach Date|Nominal Approch distance (au)|Min Close-Approach Distance (au)|Max Close-Approach Distance (au)|V Reletive (Km/s)|V Infinite (Km/s)|Close-Approach Uncertain Time|Absolute Magnitude (mag)|Diameter (Km)|Diameter-Sigma (Km)|           Object|
+-----------+--------+----------------------+-------------------+-----------------------------+--------------------------------+--------------------------------+-----------------+-----------------+-----------------------------+------------------------+-------------+-------------------+-----------------+
|   2020 BN7|       6|     2415023.594589649|1900-01-04 02:16:00|           0.0896607

#### Some Data cleaning

In [7]:
sdfl = sdfl.fillna(value=-1)
sdfl.show(2)

+-----------+--------+----------------------+-------------------+-----------------------------+--------------------------------+--------------------------------+-----------------+-----------------+-----------------------------+------------------------+-------------+-------------------+-----------------+
|Designation|Orbit Id|Time of Close approach|Close-Approach Date|Nominal Approch distance (au)|Min Close-Approach Distance (au)|Max Close-Approach Distance (au)|V Reletive (Km/s)|V Infinite (Km/s)|Close-Approach Uncertain Time|Absolute Magnitude (mag)|Diameter (Km)|Diameter-Sigma (Km)|           Object|
+-----------+--------+----------------------+-------------------+-----------------------------+--------------------------------+--------------------------------+-----------------+-----------------+-----------------------------+------------------------+-------------+-------------------+-----------------+
|   2020 BN7|       6|     2415023.594589649|1900-01-04 02:16:00|           0.0896607

In [8]:
sdfl = sdfl.withColumn('Object', trim('Object'))

#### Extracting Last Record's Date

In [13]:
lr = sdfl.tail(1)
last_date = lr[0].asDict()['Close-Approach Date']
# last_date = str(last_date.date() + timedelta(days=1))
last_date = last_date.split(' ')[0]
last_date


'2023-01-02'

In [14]:
sdfl.show(2)

+-----------+--------+----------------------+-------------------+-----------------------------+--------------------------------+--------------------------------+-----------------+-----------------+-----------------------------+------------------------+-------------+-------------------+----------+
|Designation|Orbit Id|Time of Close approach|Close-Approach Date|Nominal Approch distance (au)|Min Close-Approach Distance (au)|Max Close-Approach Distance (au)|V Reletive (Km/s)|V Infinite (Km/s)|Close-Approach Uncertain Time|Absolute Magnitude (mag)|Diameter (Km)|Diameter-Sigma (Km)|    Object|
+-----------+--------+----------------------+-------------------+-----------------------------+--------------------------------+--------------------------------+-----------------+-----------------+-----------------------------+------------------------+-------------+-------------------+----------+
|   2020 BN7|       6|     2415023.594589649|1900-01-04 02:16:00|           0.0896607474147164|           

In [9]:
# temp_df = sdfl.toPandas()



## Extracting Today's Data using API Call


In [15]:
url = "https://ssd-api.jpl.nasa.gov/cad.api"
parameters = {
    "date-min": last_date,
    "date-max": str(datetime.today().date()),
    "dist-max": "0.05",
    'fullname': "true",
    'dist-max': "0.1",
    'diameter': "true"
}
response = requests.get(url, parameters)
data = response.json()


In [16]:
data['count']

'13'

In [17]:
columns = [
    'Designation',
    'Orbit Id',
    'Time of Close approach',
    'Close-Approach Date',
    'Nominal Approch distance (au)',
    'Min Close-Approach Distance (au)',
    'Max Close-Approach Distance (au)',
    'V Reletive (Km/s)',
    'V Infinite (Km/s)',
    'Close-Approach Uncertain Time',
    'Absolute Magnitude (mag)',
    'Diameter (Km)',
    'Diameter-Sigma (Km)',
    'Object'
]
types = [
    StringType(),
    StringType(),
    DoubleType(),
    DateType(),
    DoubleType(),
    DoubleType(),
    DoubleType(),
    DoubleType(),
    DoubleType(),
    StringType(),
    DoubleType(),
    DoubleType(),
    DoubleType(),
    StringType(),
         ]
dic = {columns[i]: types[i] for i in range(len(columns))}
dic

{'Designation': StringType,
 'Orbit Id': StringType,
 'Time of Close approach': DoubleType,
 'Close-Approach Date': DateType,
 'Nominal Approch distance (au)': DoubleType,
 'Min Close-Approach Distance (au)': DoubleType,
 'Max Close-Approach Distance (au)': DoubleType,
 'V Reletive (Km/s)': DoubleType,
 'V Infinite (Km/s)': DoubleType,
 'Close-Approach Uncertain Time': StringType,
 'Absolute Magnitude (mag)': DoubleType,
 'Diameter (Km)': DoubleType,
 'Diameter-Sigma (Km)': DoubleType,
 'Object': StringType}

In [31]:
# def createSchema(columns):
#     myScehma = []
#     for key, value in dic.items():
#         t = StructField(key, value, True)
#         myScehma.append(t)
#     # print(myScehma)
#     return myScehma


# scheme = createSchema(columns)
# # print(type(scheme))
# myScehma = StructType([i for i in scheme])
# myScehma


myScehma = StructType([\
    StructField('Designation', StringType(), True),\
    StructField('Orbit Id', StringType(), True),\
    StructField('Time of Close approach', DoubleType(), True),\
    StructField('Close-Approach Date', DateType(), True),\
    StructField('Nominal Approch distance (au)', DoubleType(), True),\
    StructField('Min Close-Approach Distance (au)', DoubleType(), True),\
    StructField('Max Close-Approach Distance (au)', DoubleType(), True),\
    StructField('V Reletive (Km/s)', DoubleType(), True),\
    StructField('V Infinite (Km/s)', DoubleType(), True),\
    StructField('Close-Approach Uncertain Time', StringType(), True),\
    StructField('Absolute Magnitude (mag)', DoubleType(), True),\
    StructField('Diameter (Km)', DoubleType(), True),\
    StructField('Diameter-Sigma (Km)', DoubleType(), True),\
    StructField('Object', StringType(), True)\
])
# myScehma = StructType([\
#     StructField('Designation', StringType(), True),\
#     StructField('Orbit Id', StringType(), True),\
#     StructField('Time of Close approach', StringType(), True),\
#     StructField('Close-Approach Date', StringType(), True),\
#     StructField('Nominal Approch distance (au)', StringType(), True),\
#     StructField('Min Close-Approach Distance (au)', StringType(), True),\
#     StructField('Max Close-Approach Distance (au)', StringType(), True),\
#     StructField('V Reletive (Km/s)', StringType(), True),\
#     StructField('V Infinite (Km/s)', StringType(), True),\
#     StructField('Close-Approach Uncertain Time', StringType(), True),\
#     StructField('Absolute Magnitude (mag)', StringType(), True),\
#     StructField('Diameter (Km)', StringType(), True),\
#     StructField('Diameter-Sigma (Km)', StringType(), True),\
#     StructField('Object', StringType(), True)\
# ])
myScehma

StructType(List(StructField(Designation,StringType,true),StructField(Orbit Id,StringType,true),StructField(Time of Close approach,DoubleType,true),StructField(Close-Approach Date,DateType,true),StructField(Nominal Approch distance (au),DoubleType,true),StructField(Min Close-Approach Distance (au),DoubleType,true),StructField(Max Close-Approach Distance (au),DoubleType,true),StructField(V Reletive (Km/s),DoubleType,true),StructField(V Infinite (Km/s),DoubleType,true),StructField(Close-Approach Uncertain Time,StringType,true),StructField(Absolute Magnitude (mag),DoubleType,true),StructField(Diameter (Km),DoubleType,true),StructField(Diameter-Sigma (Km),DoubleType,true),StructField(Object,StringType,true)))

In [19]:
t = data['data']
modified_raw_data = {}
for i in range(len(columns)):
    for j in range(len(t)):
        if columns[i] in modified_raw_data:
            modified_raw_data[columns[i]].append(t[j][i])
        else:
            modified_raw_data[columns[i]] = [t[j][i]]
modified_raw_data

{'Designation': ['2022 YT3',
  '2022 YU3',
  '2022 YZ3',
  '2022 YY6',
  '2022 YP5',
  '2021 NF',
  '2022 YJ4',
  '2015 AO43',
  '2011 WR41',
  '2019 AY3',
  '2022 YS4',
  '2022 XW1',
  '2018 PN22'],
 'Orbit Id': ['7',
  '5',
  '3',
  '4',
  '6',
  '6',
  '5',
  '9',
  '11',
  '7',
  '5',
  '3',
  '6'],
 'Time of Close approach': ['2459946.529982869',
  '2459946.716108034',
  '2459946.895485196',
  '2459946.962983696',
  '2459947.244799544',
  '2459947.410115729',
  '2459947.741992184',
  '2459947.934757015',
  '2459948.221946792',
  '2459949.029916917',
  '2459949.348464181',
  '2459949.356584550',
  '2459949.488279088'],
 'Close-Approach Date': ['2023-Jan-02 00:43',
  '2023-Jan-02 05:11',
  '2023-Jan-02 09:29',
  '2023-Jan-02 11:07',
  '2023-Jan-02 17:53',
  '2023-Jan-02 21:51',
  '2023-Jan-03 05:48',
  '2023-Jan-03 10:26',
  '2023-Jan-03 17:20',
  '2023-Jan-04 12:43',
  '2023-Jan-04 20:22',
  '2023-Jan-04 20:33',
  '2023-Jan-04 23:43'],
 'Nominal Approch distance (au)': ['0.02846575

In [25]:
df = pd.DataFrame(data['data'], columns=data['fields'])
df.head()

,des,orbit_id,jd,cd,dist,dist_min,dist_max,v_rel,v_inf,t_sigma_f,h,diameter,diameter_sigma,fullname
0,2022 YT3,7,2459946.529982869,2023-Jan-02 00:43,0.0284657568010055,0.0283518611759164,0.0285796483222,6.43976919501424,6.42521761094422,< 00:01,25.814,None,None,(2022 YT3)
1,2022 YU3,5,2459946.716108034,2023-Jan-02 05:11,0.0251910694591085,0.0250168666339067,0.0253652623678832,7.13378641737377,7.11894423823365,< 00:01,25.835,None,None,(2022 YU3)
2,2022 YZ3,3,2459946.895485196,2023-Jan-02 09:29,0.0605991524261425,0.0601178894510354,0.0610804106086901,11.7046594149789,11.7009022805467,< 00:01,25.088,None,None,(2022 YZ3)
3,2022 YY6,4,2459946.962983696,2023-Jan-02 11:07,0.00543886360452723,0.00541107030146487,0.00546665622803439,20.2695510903181,20.245367587082,< 00:01,26.109,None,None,(2022 YY6)
4,2022 YP5,6,2459947.244799544,2023-Jan-02 17:53,0.0199773082677155,0.0199188916322011,0.0200357211576115,4.79371109446374,4.76580690667688,< 00:01,27.085,None,None,(2022 YP5)


In [26]:
df['jd'] = pd.to_numeric(df['jd'])
df['cd'] = pd.to_datetime(df['cd'])
df['dist'] = pd.to_numeric(df['dist'])
df['dist_min'] = pd.to_numeric(df['dist_min'])
df['dist_max'] = pd.to_numeric(df['dist_max'])
df['v_rel'] = pd.to_numeric(df['v_rel'])
df['v_inf'] = pd.to_numeric(df['v_inf'])
df['t_sigma_f'] = df['t_sigma_f'].astype(str)
df['h'] = pd.to_numeric(df['h'])
df['diameter'] = pd.to_numeric(df['diameter'])
df['diameter_sigma'] = pd.to_numeric(df['diameter_sigma'])
df.tail()

,des,orbit_id,jd,cd,dist,dist_min,dist_max,v_rel,v_inf,t_sigma_f,h,diameter,diameter_sigma,fullname
8,2011 WR41,11,2.459948e+06,2023-01-03 17:20:00,0.040538,0.040530,0.111676,8.948828,8.941480,6_22:43,25.120,NaN,NaN,(2011 WR41)
9,2019 AY3,7,2.459949e+06,2023-01-04 12:43:00,0.042987,0.018714,0.078098,19.742852,19.739712,1_14:23,23.800,NaN,NaN,(2019 AY3)
10,2022 YS4,5,2.459949e+06,2023-01-04 20:22:00,0.015371,0.015204,0.015538,6.797318,6.771768,< 00:01,25.583,NaN,NaN,(2022 YS4)
11,2022 XW1,3,2.459949e+06,2023-01-04 20:33:00,0.091095,0.090545,0.091645,6.042223,6.037380,00:07,25.175,NaN,NaN,(2022 XW1)
12,2018 PN22,6,2.459949e+06,2023-01-04 23:43:00,0.076661,0.076414,0.076908,3.782524,3.773324,02:23,27.500,NaN,NaN,(2018 PN22)


In [28]:
df.columns = columns
df.head(1)

,Designation,Orbit Id,Time of Close approach,Close-Approach Date,Nominal Approch distance (au),Min Close-Approach Distance (au),Max Close-Approach Distance (au),V Reletive (Km/s),V Infinite (Km/s),Close-Approach Uncertain Time,Absolute Magnitude (mag),Diameter (Km),Diameter-Sigma (Km),Object
0,2022 YT3,7,2.459947e+06,2023-01-02 00:43:00,0.028466,0.028352,0.02858,6.439769,6.425218,< 00:01,25.814,NaN,NaN,(2022 YT3)


In [32]:
nsdf = spark.createDataFrame(df, schema = myScehma)
nsdf


DataFrame[Designation: string, Orbit Id: string, Time of Close approach: double, Close-Approach Date: date, Nominal Approch distance (au): double, Min Close-Approach Distance (au): double, Max Close-Approach Distance (au): double, V Reletive (Km/s): double, V Infinite (Km/s): double, Close-Approach Uncertain Time: string, Absolute Magnitude (mag): double, Diameter (Km): double, Diameter-Sigma (Km): double, Object: string]

In [33]:
nsdf.show(1)

Py4JJavaError: An error occurred while calling o116.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 12.0 failed 1 times, most recent failure: Lost task 0.0 in stage 12.0 (TID 13) (VAibhAv executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:188)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:108)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:121)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:162)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:337)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RD

In [24]:
temp = [tuple(d) for d in data['data']]
temp

[('2022 YJ4',
  '5',
  '2459947.741992184',
  '2023-Jan-03 05:48',
  '0.0135605223870528',
  '0.0135184677947036',
  '0.0136025737320697',
  '5.57880952862473',
  '5.54347723391237',
  '< 00:01',
  '26.916',
  None,
  None,
  '       (2022 YJ4)'),
 ('2015 AO43',
  '9',
  '2459947.934757015',
  '2023-Jan-03 10:26',
  '0.0826054551199632',
  '0.0325049787467851',
  '0.135505833277371',
  '8.46115194400779',
  '8.45733889836805',
  '9_07:59',
  '26.5',
  None,
  None,
  '       (2015 AO43)'),
 ('2011 WR41',
  '11',
  '2459948.221946792',
  '2023-Jan-03 17:20',
  '0.0405377113458668',
  '0.0405302257042255',
  '0.111676045470241',
  '8.94882792980961',
  '8.94147999472359',
  '6_22:43',
  '25.12',
  None,
  None,
  '       (2011 WR41)')]

In [26]:
rdd = spark.sparkContext.parallelize(temp)


In [27]:
nsdfrdd = rdd.toDF(columns)
nsdfrdd.printSchema()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 1 times, most recent failure: Lost task 0.0 in stage 8.0 (TID 9) (VAibhAv executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:164)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.spark.scheduler.Ta

In [54]:
# nsdf = spark.createDataFrame(modified_raw_data, schema=myScehma)
# nsdf

In [ ]:
df = pd.DataFrame(np.array(data['data']), columns = data['fields'])
df.head()

In [ ]:
df['jd'] = pd.to_numeric(df['jd'])
df['cd'] = pd.to_datetime(df['cd'])
df['dist'] = pd.to_numeric(df['dist'])
df['dist_min'] = pd.to_numeric(df['dist_min'])
df['dist_max'] = pd.to_numeric(df['dist_max'])
df['v_rel'] = pd.to_numeric(df['v_rel'])
df['v_inf'] = pd.to_numeric(df['v_inf'])
df['h'] = pd.to_numeric(df['h'])
df['diameter'] = pd.to_numeric(df['diameter'])
df['diameter_sigma'] = pd.to_numeric(df['diameter_sigma'])
df.tail()

In [ ]:
df = pd.read_csv('./raw_input_data.csv')
df

In [ ]:
df.columns = columns
df

In [ ]:
df.to_csv('./raw_input_data.csv', index=False)

In [ ]:
# psdf = ps.DataFrame(np.array(data['data']))
# psdf.head()

In [ ]:
sdf = spark.read.csv('./raw_input_data.csv', inferSchema=True, header=True)
sdf

In [ ]:
sdf.show(2)

In [ ]:
sdf = sdf.fillna(value=-1)

In [ ]:
sdf.show(2)

In [ ]:
# psdf = ps.DataFrame(
#     {'a': [1, 2, 3, 4, 5, 6],
#      'b': [100, 200, 300, 400, 500, 600],
#      'c': ["one", "two", "three", "four", "five", "six"]})

In [ ]:
df.columns = columns
df

In [ ]:
# with open('data.json', 'w') as datafile:
#     json.dump(data['data'], datafile)

In [ ]:
# psdf = ps.from_pandas(df)

In [ ]:
# sdf = spark.read.schema(myScehma).json('./data.json')
# sdf.printSchema()

sdf = spark.createDataFrame(df)


In [ ]:
sdf.show()

In [ ]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
sdf = spark.createDataFrame(df)


In [ ]:
sdf.printSchema()


In [ ]:

import pandas as pd    
data = [['Scott', 50], ['Jeff', 45], ['Thomas', 54],['Ann',34]] 
  
# Create the pandas DataFrame 
pandasDF = pd.DataFrame(data, columns = ['Name', 'Age']) 
  
# print dataframe. 
print(pandasDF)

# from pyspark.sql import SparkSession

# spark = SparkSession.builder \
#     .master("local[1]") \
#     .appName("SparkByExamples.com") \
#     .getOrCreate()

sparkDF=spark.createDataFrame(pandasDF) 
sparkDF.printSchema()
sparkDF.show()

#sparkDF=spark.createDataFrame(pandasDF.astype(str)) 
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
mySchema = StructType([ StructField("First Name", StringType(), True)\
                       ,StructField("Age", IntegerType(), True)])

sparkDF2 = spark.createDataFrame(pandasDF,schema=mySchema)
sparkDF2.printSchema()
sparkDF2.show()

# Enable Apache Arrow to convert Pandas to PySpark DataFrame
spark.conf.set("spark.sql.execution.arrow.enabled","true")
sparkDF2=spark.createDataFrame(pandasDF) 
sparkDF2.printSchema()
sparkDF2.show()

#Convert PySpark DataFrame to Pandas
pandasDF2=sparkDF2.select("*").toPandas
print(pandasDF2)



In [ ]:
sdf.show(5)

In [ ]:
lunar_distance_multiplier = 389.174